# Sampling from symmetric Dirichlet distribution priors

## Jupyter notebook tips

### Equations

This Markdown cell has **equations**, written using [*LaTeX*](http://www.latex-project.org/) syntax and rendered in the browser using [*MathJax*](http://www.mathjax.org/) (which [the Cornell Library helped develop!](http://news.library.cornell.edu/news/110104/mathjax)).

For help with LaTeX/MathJax math syntax, see:
* [Math Examples — Jupyter Notebook documentation](http://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Typesetting%20Equations.html)
* [A Primer on Using LaTeX in Jupyter Notebooks | Democratizing Data](http://data-blog.udacity.com/posts/2016/10/latex-primer/) — A brief blog post introducing essential notation (Bayes's theorem is an example!)
* A short [MathJax tutorial](https://math.meta.stackexchange.com/questions/5020/mathjax-basic-tutorial-and-quick-reference) at the StackExchange MathematicsMeta site
* A two-page [Quick guide to LaTeX](http://divisbyzero.com/2011/08/17/a-quick-guide-to-latex/) (PDF; useful mainly just for the list of symbols)
* A [gentle intro the basics at FluidMind.org](http://fluidmind.org/articles/typesetting-equations/) (just go through the first three sections, through "Placing Equations on Web Pages")
* The [*AMS Short Math Guide for LaTeX*](https://ctan.org/tex-archive/info/short-math-guide?lang=en) (PDF file); note that not everything described here is implemented in MathJax
* The Harvard math department hosts [An introduction to Using TeX](http://www.math.harvard.edu/texman/); for MathJax use, the "Commands for Math Mode" is most relevant

Note that equation support is an *extension* to basic Markdown, and not fully standardized.  If you are using a standalone Markdown editor/previewer, or a different browser-based Markdown editor (such as [StackEdit](https://stackedit.io/)), be aware that there are different conventions for how to set off inline and displayed equations in extended Markdown:
* [Survey of syntaxes for math in markdown](https://github.com/cben/mathdown/wiki/math-in-markdown)

#### Equation examples

Here is *Bayes's theorem* for a posterior distribution over a set of hypotheses $\{H_i\}$ based on observed data $D$:
$$
p(H_i|D) = \frac{p(H_i) p(D|H_i)}{p(D)} \qquad ||~\mathcal{C}.
$$
And, of course, the all-important *LTP*, where $\{B_j\}$ is a *suite* (i.e., a mutually exclusive, exhaustive set of alternatives):
$$
p(H_i) = \sum_j p(H_i,B_j) = \sum_j p(B_j) p(H_i|B_j) \qquad ||~\mathcal{C}.
$$
And since it's so important, let's write the LTP with alignment (using the LaTeX *align* environment):
\begin{align}
p(H_i)
  &= \sum_j p(H_i,B_j)\\
  &= \sum_j p(B_j) p(H_i|B_j) \qquad ||~\mathcal{C}.
\end{align}

Finally, here's the *generalized beta integral*, from the lecture on categorical/multinomial inference (I'll use a *split* environment, so I can break the long equation):
\begin{split}
\int_0^\infty d\alpha_1 \cdots \int_0^\infty d\alpha_K\;
  & % ampersand indicates align here
  \alpha_1^{\kappa_1-1}\cdots\alpha_K^{\kappa_K-1} \delta\left(a-\sum_k\alpha_k\right)\\  % split!
  &= \frac{\Gamma(\kappa_1)\cdots\Gamma(\kappa_K)}{\Gamma(\kappa_0)}\; a^{\kappa_0-1}
\end{split}
where $\kappa_0 = \sum_{k=1}^K \kappa_k$.

### Tables

This Markdown cell has a table.  To automatically generate nice-looking Markdown table markup, I used
[TablesGenerator](http://www.tablesgenerator.com/markdown_tables).  This table could hold ingredients for calculating the posterior PMF for the Monty Hall problem:

| Hypothesis | Prior | Likelihood | Prior $\times$ Likelihood | Posterior |
|------------|-------|------------|---------------------------|-----------|
| A          |       |            |                           |           |
| B          |       |            |                           |           |
| C          |       |            |                           |           |
| Sum        | 1     | NA         |                           |     1     |

---

## Symmetric Dirichlet distributions

One compelling criterion for a categorical prior that is uninformative about the categorical PMF is that the prior not express any preference for some categories over others.  This requires that the prior be *symmetric* with respect to the $\alpha_k$ category probability parameters.

The Dirichlet distribution is a PDF for the $\alpha_k$ parameters describing a categorical PMF.  It has $K$ concentration parameters, $\kappa_k$. The Dirichlet PDF is given by
$$
p(\alpha_1,\ldots,\alpha_K) =
  \frac{\Gamma(\kappa_0)}{\Gamma(\kappa_1)\cdots\Gamma(\kappa_K)}\;
  \alpha_1^{\kappa_1-1}\cdots\alpha_K^{\kappa_K-1}\;
  \delta\left(1-\sum_k\alpha_k\right),
$$
where $\kappa_0 = \sum_{k=1}^K \kappa_k$. 

A *symmetric* Dirichlet distribution is a Dirichlet distribution with all concentration parameters equal to each other: $\kappa_i = \kappa$:
$$
p(\alpha_1,\ldots,\alpha_K) =
  \frac{\Gamma(K\cdot\kappa)}{[\Gamma(\kappa)]^K}\;
  \alpha_1^{\kappa-1}\cdots\alpha_K^{\kappa-1}\;
  \delta\left(1-\sum_k\alpha_k\right),
$$
If $\kappa = 1$, the resulting symmetric Dirichlet PDF is constant—the prior used in Lec06. Here we'll explore this prior, and an alternative.

In [ ]:
from matplotlib.pyplot import *
from scipy import *
from scipy.stats import dirichlet

from shelves import shelves

# pyplot.ion() tells Jupyter to make plots in cells, or tells IPython
# to make plots in an interactive window.
ion()

In [ ]:
class SymmetricDirichlet:
    
    def __init__(self, ncat, kappa):
        """
        Define a symmetric Dirichlet distribution over PMFs with `ncat` categories,
        with each category's concentration parameter equal to `kappa`.
        """
        self.ncat = ncat
        self.kappa = kappa
        self.kappa0 = ncat*kappa
        self.kappas = kappa*ones(ncat)
        
        # Define the Dirichlet by setting all kappas = kappa.
        self.distn = dirichlet(self.kappas)

    def pdf(self, alphas):
        """
        Compute the PDF for category probabilities given by `alphas`.
        """
        return self.distn.pdf(alphas)

    def sample(self, n=1):
        """
        Return `n` samples from a symmetric Dirichlet.
        """
        return self.distn.rvs(n)

### Uniform symmetric Dirichlet priors

In [ ]:
flatsd2 = SymmetricDirichlet(2, 1.)
samps = flatsd2.sample(10)

In [ ]:
samps

In [ ]:
def plot_samples(samples, delta=1.2):
    """
    Make a stack of plots of the PMFs in `samples`, each one offset
    vertically by `delta`.
    """
    figure(figsize=(10,7))
    dy = 0.  # starting shift
    for samp in samples:
        shelves(samp, dy=dy)  # imported from shelves.py
        axhline(dy, c='k')
        dy += delta

    xlim(0., 1.)

In [ ]:
plot_samples(samps)

In [ ]:
flatsd5 = SymmetricDirichlet(5, 1.)
samps = flatsd5.sample(10)
plot_samples(samps)

In [ ]:
flatsd20 = SymmetricDirichlet(20, 1.)
samps = flatsd20.sample(10)
plot_samples(samps)

In [ ]:
plot_samples(samps, delta=.5)

To understand this behavior, recall the single-category marginal implied by a Dirichlet distribution.  E.g., in Lec06, we found that for three categories, the marginal posterior for $\alpha_1$ when we use a flat prior and see $(n_1, n_2, n_3)$ counts (with $N=n_1+n_2+n_3$) is
\begin{align}
p(\alpha_1|D)
  &= \int d\alpha_2 \int d\alpha_3\; p(\alpha_1,\alpha_2,\alpha_3|D)\\
  &\propto \alpha_1^{n_1} \int d\alpha_2 \int d\alpha_3\; 
      \alpha_2^{n_2}\alpha_3^{n_3}
      \; \delta\left[(1-\alpha_1) - (\alpha_2 + \alpha_3)\right]\\
  &\propto \alpha_1^{n_1} (1-\alpha_1)^{n_2+n_3}; \qquad \mbox{note } n_2+n_3 = N-n_1.
\end{align}
What does this imply for a many-category flat prior?

---

### Non-uniform symmetric Dirichlet priors

In [ ]:
# "ac" for "aggregation consistent" - see exercise!
acsd5 = SymmetricDirichlet(5, 1./5)
samps = acsd5.sample(10)
plot_samples(samps)

In [ ]:
acsd20 = SymmetricDirichlet(20, 1./20)
samps = acsd20.sample(10)
plot_samples(samps)

In [ ]:
# Something in between:
sd20 = SymmetricDirichlet(20, 1./sqrt(20))
samps = sd20.sample(10)
plot_samples(samps)